<a href="https://colab.research.google.com/github/zzhining/ml_basic/blob/main/CNN_BASIC_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습 00 - Basic Conv Net

## 실습내용
-----
- AlexNet을 사용하여 이미지를 학습하고 10개의 카테고리를 갖는 이미지를 분류하는 이미지 분류기를 생성합니다.
- 데이터셋: [MNIST](https://pytorch.org/vision/0.9/datasets.html#mnist)


------
**[reference]**
- https://tutorials.pytorch.kr/beginner/basics/buildmodel_tutorial.html
- https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html
- https://medium.com/@djin31/how-to-plot-wholesome-confusion-matrix-40134fd402a8


## [Step1] 라이브러리 및 데이터 불러오기

**[torch.nn](https://pytorch.org/docs/stable/nn.html)** :  신경망을 생성하기 위한 기본 재료들을 제공(Modules, Sequential, Layer, Activations, Loss, Dropout...)



**[torchvision.datasets](https://pytorch.org/vision/0.9/datasets.html#fashion-mnist)** : torchvision.transforms를 사용해 변형이 가능한 형태, feature와 label을 반환

**[torchvision.transforms](https://tutorials.pytorch.kr/beginner/basics/transforms_tutorial.html)**

* ToTensor() : ndarray를 FloatTensor로 변환하고 이미지 픽셀 크기를 [0., 1.]범위로 조정(scale)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch import nn

from torchvision import datasets
from torchvision.transforms import transforms
from torchvision.transforms.functional import to_pil_image

In [ ]:
%load_ext tensorboard
from torch.utils.tensorboard import SummaryWriter

### MNIST 데이터 불러오기

In [ ]:
train_img = datasets.MNIST(
    root = 'data',
    train = True,
    download = True,
    transform = transforms.ToTensor(),
)

test_img = datasets.MNIST(
    root = 'data',
    train = False,
    download = True,
    transform = transforms.ToTensor(),
)

### 하이퍼파라미터 셋팅

In [ ]:
# 하이퍼파라미터 준비
EPOCH = 10
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", DEVICE)

writer = SummaryWriter('log')

### DataLoader 만들기

In [ ]:
train_loader = DataLoader(train_img, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(test_img, batch_size = BATCH_SIZE, shuffle = False)

## [Step2] EDA, 데이터 전처리

In [ ]:
figure = plt.figure(figsize = (8, 8))
cols, rows = 5, 5

for i in range(1, cols * rows +1):
    sample_idx = torch.randint(len(train_img), size=(1,)).item()
    img, label = train_img[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.axis('off')
    plt.imshow(to_pil_image(img), cmap='gray')
plt.show()

## [Step3] 모델 생성 및 학습

#### Model Calss 만들기

In [ ]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(128 * 3 * 3, 2),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2),            
        )

        self.output = nn.Sequential(
            nn.Linear(2, 10),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = self.output(x)      
        return x

#### Model instance 생성

In [ ]:
model = ConvNet().to(DEVICE)
print(model)

#### Loss Function

In [ ]:
loss = nn.CrossEntropyLoss()

#### Optimizer

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

#### Training

In [ ]:
def train(train_loader, model, loss_fn, optimizer):
    model.train()
    
    size = len(train_loader.dataset)
    num_batches = len(test_loader)
    train_loss, correct = 0, 0

    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(DEVICE), y.to(DEVICE)
        _, pred = model(X)
        
        # 손실 계산
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f'loss: {loss:>7f}  [{current:>5d}]/{size:5d}')  

    train_loss /= num_batches
    correct /= size

    return train_loss, correct

#### Test

In [ ]:
def test(test_loader, model, loss_fn):
    feature_list = []
    label_list = []
    model.eval()

    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            _, pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            feature_list.append(pred)
            label_list.append(y)

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")

    feats = torch.cat(feature_list, 0)
    labels = torch.cat(label_list, 0)
    return test_loss, correct

#### 학습 진행하기

In [ ]:
for i in range(EPOCH) :
    print(f"Epoch {i+1} \n------------------------")
    train_loss, train_acc = train(train_loader, model, loss, optimizer)
    test_loss, test_acc = test(test_loader, model, loss)

    epoch = i+1
    writer.add_scalar('Train Loss', train_loss, epoch)
    writer.add_scalar('Train Acc', train_acc*100, epoch)
    writer.add_scalar('Test Loss', test_loss, epoch)
    writer.add_scalar('Test acc', test_acc*100, epoch)
print("Done!")

In [ ]:
writer.close()
%tensorboard --logdir 'log'